In [13]:
# Getting CSV File from that Github Data:
import requests
import sqlite3
import pandas as pd
import os
from tqdm import tqdm # Indication of Download

download_urls = []
URL = "https://api.github.com/repos/CSSEGISandData/COVID-19/contents/csse_covid_19_data/csse_covid_19_daily_reports"
response = requests.get(URL)
for data in tqdm(response.json()):
	if data['name'].endswith('.csv'):
		download_urls.append(data['download_url'])
download_urls[:10]

100%|██████████| 529/529 [00:00<00:00, 412490.58it/s]


['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-01-2021.csv',
 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-02-2021.csv',
 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-03-2021.csv',
 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-04-2021.csv',
 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-05-2021.csv',
 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-06-2021.csv',
 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-07-2021.csv',
 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_da

In [5]:
# Labels that want or need to renamed :
relabel = {
    'Country/Region': 'Country_Region',
    'Lat': 'Latitude',
    'Long_': 'Longitude',
    'Province/State': 'Province_State',
}

In [6]:
# Function to rename 
def factor_dataframe(dat, filename):
    """ Refactor the dataframe to be uploaded into a SQL database
    as a pandas DataFrame
    """
    # rename labels
    for label in dat:
        if label in relabel:
            dat = dat.rename(columns = {label: relabel[label]})
    
    # return a dataframe with these parameters
    labels = ['Province_State', 'Country_Region', 'Last_Update', 'Confirmed', 'Deaths', 'Recovered']
    # filename is datetime
    if 'Last_Update' not in dat:
        dat['Last_Update'] = pd.to_datetime(filename)

    # replace columns not in dataframe with nan
    for label in labels:
        if label not in dat:
            dat[label] = np.nan

    return dat[labels]

In [19]:
for i,file_path in tqdm(list(enumerate(download_urls))):
    print(file_path)
df = pd.read_csv(file_path)
df.head() # we only want our selected data 
factor_dataframe(df,download_urls)

100%|██████████| 527/527 [00:00<00:00, 6103.66it/s]


https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-01-2021.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-02-2021.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-03-2021.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-04-2021.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-05-2021.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-06-2021.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-07-2021.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,2021-01-01 05:23:07,51526,2191,41727
1,NaN,Albania,2021-01-01 05:23:07,58316,1181,33634
2,NaN,Algeria,2021-01-01 05:23:07,99610,2756,67127
3,NaN,Andorra,2021-01-01 05:23:07,8049,84,7432
4,NaN,Angola,2021-01-01 05:23:07,17553,405,11044
...,...,...,...,...,...,...
3979,Unknown,China,2021-01-01 05:23:07,0,0,0
3980,NaN,Kiribati,2021-01-01 05:23:07,0,0,0
3981,NaN,Palau,2021-01-01 05:23:07,0,0,0
3982,Cook Islands,New Zealand,2021-01-01 05:23:07,0,0,0


In [7]:
def upload_to_sql(filenames, db_name, debug=False):
    """ Given a list of paths, upload to a database
    """
    conn = sqlite3.connect(f"{db_name}.db")
    
    if debug:
        print("Uploading into database")
    for i, file_path in tqdm(list(enumerate(filenames))):
        
        dat = pd.read_csv(file_path)

        # rename labels
        filename = os.path.basename(file_path).split('.')[0]
        dat = factor_dataframe(dat, filename)

        # write records to sql database
        if i == 0: # if first entry, and table name already exist, replace
            dat.to_sql(db_name, con=conn, index = False, if_exists='replace')
        else: # otherwise append to current table given db_name
            dat.to_sql(db_name, con=conn, index = False, if_exists='append')


In [14]:
upload_to_sql(download_urls, 'example', debug=True)

  0%|          | 0/527 [00:00<?, ?it/s]

Uploading into database


100%|██████████| 527/527 [09:20<00:00,  1.06s/it]


In [29]:
conn = sqlite3.connect('example.db')
print("Opened database successfully")
conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute(" SELECT country_region, SUM(Confirmed), DATE(last_update) AS last_update FROM example GROUP BY country_region, last_update ORDER BY last_update;")
c.fetchall()

Opened database successfully


[('Afghanistan', 34, None),
 ('Afghanistan', 107, None),
 ('Afghanistan', 118, None),
 ('Afghanistan', 146, None),
 ('Afghanistan', 240, None),
 ('Afghanistan', 300, None),
 ('Afghanistan', 368, None),
 ('Albania', 89, None),
 ('Albania', 197, None),
 ('Albania', 212, None),
 ('Albania', 223, None),
 ('Albania', 277, None),
 ('Albania', 333, None),
 ('Albania', 377, None),
 ('Algeria', 201, None),
 ('Algeria', 454, None),
 ('Algeria', 511, None),
 ('Algeria', 584, None),
 ('Algeria', 986, None),
 ('Algeria', 1251, None),
 ('Algeria', 1423, None),
 ('Andorra', 113, None),
 ('Andorra', 308, None),
 ('Andorra', 334, None),
 ('Andorra', 370, None),
 ('Andorra', 428, None),
 ('Andorra', 466, None),
 ('Andorra', 525, None),
 ('Angola', 2, None),
 ('Angola', 5, None),
 ('Angola', 7, None),
 ('Angola', 7, None),
 ('Angola', 8, None),
 ('Angola', 10, None),
 ('Angola', 16, None),
 ('Antigua and Barbuda', 1, None),
 ('Antigua and Barbuda', 7, None),
 ('Antigua and Barbuda', 7, None),
 ('Antigua 